<a href="https://colab.research.google.com/github/mfligiel/Models-for-MLOPS-Review/blob/main/Evidently_for_WeatherModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Weather Data

I am going to predict Chicago's weather from the weather of 5 other places nearby using a weather API.  This model isn't the most useful, but is good for showcasing model monitoring.

Here, I will pull in some June data, but replacing Toronto with Phoenix.  A bit of a different temperature distribution!

In [1]:
!pip install evidently

     |████████████████████████████████| 15.2MB 185kB/s 


In [2]:
import requests
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import evidently
import pickle

In [3]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


This should work!  I'll now find the IDs of 5 cities I will use to predict Chicago's weather:

Milwaukee\
Detroit\
Toronto\
St Louis\
Omaha, NE


I'll use this site to look it up: https://www.findmecity.com/

Milwaukee: 2451822\
Detroit: 2391585 \
Toronto: 4118\
St. Louis: 2486982\
Omaha, NE: 2465512

I'll switch Toronto's WOEID for that of Phoenix: 2471390


In [4]:
#dictionary of cities
cities = {'Milwaukee':'2451822', 'Detroit':'2391585', 'Toronto':'2471390', 'St. Louis':'2486982', 'Omaha':'2465512', 'Chicago':'2379574'} #phoenix for toronto

#empty list to enter these into:
values = []

#loop through cities
for k, v in cities.items():
  #loop through 3 months
  for mth in ['6']:
    #just do days through 30, it's not time series, I don't care
    for day in range(1, 15):
      #what to request
      strng = 'https://www.metaweather.com/api/location/' + v +'/2021/' + mth + '/' +str(day) + '/'
      if day == 1:
        print(strng)
      reqst = requests.get(strng)
      #get the pieces
      date = pd.to_datetime(pd.DataFrame(reqst.json()).max()['created']).date()
      maxtemp = pd.DataFrame(reqst.json()).max()['max_temp']
      values.append([k, date, maxtemp])
      time.sleep(3)





https://www.metaweather.com/api/location/2451822/2021/6/1/
https://www.metaweather.com/api/location/2391585/2021/6/1/
https://www.metaweather.com/api/location/2471390/2021/6/1/
https://www.metaweather.com/api/location/2486982/2021/6/1/
https://www.metaweather.com/api/location/2465512/2021/6/1/
https://www.metaweather.com/api/location/2379574/2021/6/1/


In [5]:
import pickle

#pickle.dump(values, open('.pkl', 'wb'))

In [6]:
pd.DataFrame(values).to_csv('Test.csv')

In [7]:
!ls

gdrive	sample_data  Test.csv


In [8]:
!cp Test.csv gdrive/MyDrive

## Loading in this data (once saved), and the model

Now I can prep to predict, and see how different it really is.

In [ ]:
#load in data
df = pd.read_csv()
#load in model
model = pickle.load(open('', 'rb'))

In [ ]:
#Now, to rename the columns
df.columns = ['drp', 'city', 'date', 'maxtemp']
df.drop('drp', axis=1, inplace=True)

## Evidently